# **Parking Information System results**

In [ ]:
import numpy as np
import pandas as pd
import os
import json
import matplotlib.pyplot as plt

import visualization
from importlib import reload

reload(visualization);

In [ ]:
MEAS_ROOT = "../02_data/"
MEAS_CASES = ["mix10", "mix25", "mix50"]
SEEDS = [42, 1812, 9698, 424, 820, 75, 98, 65535, 16383, 513]
PENS = [0, 2, 4, 6, 8, 10]
PENS = [0, 2, 6, 10]

In [ ]:
for mc in MEAS_CASES:
    for s in SEEDS:
        if not os.path.exists(f"{MEAS_ROOT}/{mc}/information0/"):
            os.makedirs(f"{MEAS_ROOT}/{mc}/information0/")
        os.remove(f"{MEAS_ROOT}/{mc}/information0/{s}")
        os.symlink(
            src = f"../../{mc}/auction0/{s}",
            dst = f"{MEAS_ROOT}/{mc}/information0/{s}",
            target_is_directory=True
        )

## Occupancy

In [ ]:
def combine_occups(meas_series, meas_name, seeds):
    meas_path = f"{MEAS_ROOT}/{meas_series}/{meas_name}"
    occupancies = pd.DataFrame()
    for seed in seeds:
        occup_df = pd.read_csv(f"{meas_path}/{seed}/occupancy.csv")
        occup_df["seed"] = seed
        occupancies = pd.concat([occupancies, occup_df])
    return occupancies

In [ ]:
def plot_occupancy(occup_df, fig=None, ax=None, figsize=(10,12)):
    occup_filt = occup_df #no warning
    occup_filt["parking_id"] = occup_df["parking_id"].str.strip("pa")
    
    mean_occupancy = occup_filt.groupby("parking_id").mean()["occupancy"]
    occup_plot = {}
    for pid in mean_occupancy.index:
        occup_plot[pid] = mean_occupancy[pid]
    
    if ax is None:
        fig, ax = visualization.plot_network_probs(
            "../01_simulation/02_scenario/grid.net.xml",
            occup_plot,
            cmap="RdBu", #p_min = 0.0,  p_max=1.2
        )
    else:
        visualization.plot_network_probs(
            "../01_simulation/02_scenario/grid.net.xml",
            occup_plot,
            cmap="RdBu", #p_min = 0.0,  p_max=1.2
            fig=fig, ax=ax, figsize=figsize
        )
    return fig, ax

In [ ]:
for i, pen in enumerate(PENS):
    for j, case in enumerate(MEAS_CASES):
        occup = combine_occups(case, f"information{pen}", SEEDS)
        plot_occupancy(occup, figsize=(5,6))
        plt.savefig(f"ax_{i}_{j}.png")
        plt.close()
        
fig, ax = plt.subplots(
    ncols = 3, nrows = 4,
    figsize = (3*5+.1, 4*5+.1)
)

for i in range(len(PENS)):
    for j in range(len(MEAS_CASES)):
        with open(f"ax_{i}_{j}.png",'rb') as f:
            image=plt.imread(f)
            ax[i][j].get_xaxis().set_ticks([])
            ax[i][j].get_yaxis().set_ticks([])
            for spine in ax[i, j].spines.values():
                spine.set_visible(False)
            if i==0:
                ax[i][j].set_title(f"{MEAS_CASES[j]}")
            if j==0:
                ax[i][j].set_ylabel(f"Penetration: {PENS[i]*10}%")
            ax[i][j].imshow(image)
            
        os.remove(f"ax_{i}_{j}.png")
fig.tight_layout()

## Vehicle data

In [ ]:
def read_vehicle_data(meas_series, meas_name, seeds, col_names, col_values):
    meas_path = f"{MEAS_ROOT}/{meas_series}/{meas_name}"
    veh_data_comb = pd.DataFrame()
    for seed in seeds:
        with open(f"{meas_path}/{seed}/veh_results.json") as f:
            veh_res = json.load(f)
        veh_data = pd.DataFrame.from_dict(veh_res, orient="index")
        veh_data["seed"] = seed
        veh_data.index = veh_data.index.astype(int)

        trip_df = pd.read_xml(f"{meas_path}/{seed}/vehicle_trips.xml", xpath = "tripinfo")
        trip_df["id"] = trip_df["id"].astype(int)

        vd = pd.merge(left=trip_df, right=veh_data, left_on="id", right_index=True)
        
        for col_name, col_value in zip(col_names, col_values):
            vd[col_name] = [col_value]*len(vd)
        
        veh_data_comb = pd.concat([veh_data_comb, vd])
        #print(veh_data_comb.columns)
    
    #veh_data_comb["prolonged_distance"] = np.clip(veh_data_comb["routeLength"] - veh_data_comb["original_distance"], a_min = 0, a_max=None)
    return veh_data_comb

In [ ]:
def get_mean_values(veh_data):
    return {
        "avg_paid_price": np.mean(veh_data["paid_price"]),
        "avg_prolonged_distance": np.mean(veh_data["parking_distance"]),
        "avg_total_parking_revenue": np.mean(veh_data.groupby("seed").sum()["paid_price"]),
        "avg_n_reroutes": np.mean(veh_data["rerouteNo"])
    }

In [ ]:
vehicle_data = pd.DataFrame()
for meas_case in MEAS_CASES:
    for pen in PENS:
        vd_ = read_vehicle_data(meas_case, f"information{pen}", SEEDS,
                                ["mix", "penetration"], [meas_case, pen])
        vehicle_data = pd.concat([vehicle_data, vd_])

In [ ]:
vehicle_data.head()

In [ ]:
#reroutes
for meas_case in MEAS_CASES:
    plot_points_y = []
    for pen in PENS:
        vd_filt = vehicle_data[
            np.logical_and(vehicle_data["mix"] == meas_case,
                           vehicle_data["penetration"] == pen)
        ]
        plot_points_y.append(vd_filt["rerouteNo"].mean())
    plt.plot(np.array(PENS)*10, plot_points_y, label=meas_case)
    plt.scatter(np.array(PENS)*10, plot_points_y)
    
plt.legend()
plt.xlabel("penetration [%]")
plt.ylabel("avg. #reroutes")
plt.title("information system #reroutes")

In [ ]:
#reroutes
for meas_case in MEAS_CASES:
    plot_points_y = []
    for pen in PENS:
        vd_filt = vehicle_data[
            np.logical_and(vehicle_data["mix"] == meas_case,
                           vehicle_data["penetration"] == pen)
        ]
        plot_points_y.append(vd_filt["parking_price"].mean())
    plt.plot(np.array(PENS)*10, plot_points_y, label=meas_case)
    plt.scatter(np.array(PENS)*10, plot_points_y)
    
plt.legend()
plt.xlabel("penetration [%]")
plt.ylabel("avg. parking price [€]")
plt.title("information system price")

In [ ]:
#reroutes
for meas_case in MEAS_CASES:
    plot_points_y = []
    for pen in PENS:
        vd_filt = vehicle_data[
            np.logical_and(vehicle_data["mix"] == meas_case,
                           vehicle_data["penetration"] == pen)
        ]
        plot_points_y.append(vd_filt["parking_distance"].mean())
    plt.plot(np.array(PENS)*10, plot_points_y, label=meas_case)
    plt.scatter(np.array(PENS)*10, plot_points_y)
    
plt.legend()
plt.xlabel("penetration [%]")
plt.ylabel("avg. parking distance [m]")
plt.title("information system distance")

## Traffic flow

In [ ]:
def read_flow(meas_series, meas_name, seeds):
    flow_series = []
    for seed in SEEDS:
        flow_df = pd.read_xml(f"{MEAS_ROOT}/{meas_series}/{meas_name}/{seed}/detector_data.out.xml", xpath="interval")
        flow_series.append(flow_df.groupby("begin")["flow"].mean().values[:20])
    #for fl in flow_series:
        #print(len(fl))
    flows = np.array(flow_series)
    return flows

In [ ]:
for meas_case in MEAS_CASES:
    for pen in PENS:
        flows = read_flow(meas_case, f"information{pen}", SEEDS)
        flow = np.mean(flows, axis=0)
        plt.plot(np.arange(0, 15*4*5, 15), flow, label=f"{pen*10}%")
    
    plt.legend(title="penetration")
    plt.xlabel("time [min]")
    plt.ylabel("avg. flow [veh/h]")
    plt.ylim(100, 160)
    plt.title(f"traffic flow\ninformation system {meas_case}")
    plt.show()